# Preprocessing
This script loads the images and crops them down multiple times for each cell, that is contained in the image.

In [ ]:
import numpy as np
import os
from pathlib import Path
from PIL import Image
import json
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchvision import transforms as T
import torch
import random
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [ ]:
# ------------  CONFIG  -------------------
base_path = '/scratch/cv-course-group-5/data/dataset_jpg'
TARGET_SIZE = 224
src_root   = Path(base_path + '/dataset')
dst_root   = Path(base_path + '/preprocessed_dataset')
lmdb_path = Path(base_path + '/lmdb')
anno_file  = Path(base_path + '/dataset/annotations.json')

## Crop images

In [ ]:
# load annotations
annos_dict = json.loads(anno_file.read_text())

annos = annos_dict.get('annotations', [])
videos = annos_dict.get('videos', [])
images = annos_dict.get('images', [])

video_id2name = {v["id"]: v["name"] for v in videos}
image_by_id = {img["id"]: img for img in images} 

In [ ]:
# reorder annotations in new dictionary
annos_sorted = {}

for ann in annos:
    video_id = ann["video_id"]
    image_id = ann["image_id"]

    frame_nmbr = image_by_id[image_id]['file_name'][-7:-4]
    video_name = video_id2name[video_id]
    
    # if there is no entry yet in the dictionary
    if video_name not in annos_sorted:
        annos_sorted[video_name] = {}
    if frame_nmbr not in annos_sorted[video_name]:
        annos_sorted[video_name][frame_nmbr] = []

    # add annotation
    annos_sorted[video_name][frame_nmbr].append(ann)

In [ ]:
video_folder_list = os.listdir(src_root)
video_folder_list = [f for f in video_folder_list if f != 'annotations.json']
video_folder_list = sorted(video_folder_list, key=lambda x: int(x))

In [ ]:
# missing folder names in annotations (written manually)
missing_folders = ['052', '060', '160', '256', '265']

In [ ]:
import lmdb
import cv2
from io import BytesIO

map_size = 1 << 40

env = lmdb.open(str(lmdb_path), map_size=map_size)

keys = []

with env.begin(write=True) as txn:
    for _, vid_name in tqdm(enumerate(video_folder_list), total=len(video_folder_list), ncols=100, position=0):

        # define paths for specific video folder
        org_video_folder = src_root / vid_name / 'images'

        try:
            annos_by_frame = annos_sorted[vid_name]
        except KeyError:
            print(vid_name)
            continue

        # iterate over every image file in that folder
        for img_path in sorted(org_video_folder.glob("*.jpg")):
            # frame index is the last 3 digits in “…t086.jpg”
            frame_idx = img_path.stem[-3:]

            # skip if no cells in this frame
            if frame_idx not in annos_by_frame:
                print(img_path)
                continue

            img = Image.open(img_path).convert("RGB")

            for ann in annos_by_frame[frame_idx]:
                cell_id = str(ann['cell_id'])
                x_c, y_c, w, h = ann['bbox']

                # calculate values for cropping
                left = int(x_c - w / 2)
                upper = int(y_c - h / 2)
                right = int(x_c + w / 2)
                lower = int(y_c + h / 2)

                crop = img.crop((left, upper, right, lower))

                # resize image if it is larger than the Target size
                if crop.width > TARGET_SIZE or crop.height > TARGET_SIZE:
                    scale = TARGET_SIZE / max(crop.width, crop.height)
                    new_size = (int(crop.width * scale), int(crop.height * scale))
                    crop = crop.resize(new_size, Image.BICUBIC)

                # put image on black background of Target size
                canvas = Image.new("RGB", (TARGET_SIZE, TARGET_SIZE), (0, 0, 0))  # black background
                x_off = (TARGET_SIZE - crop.width) // 2
                y_off = (TARGET_SIZE - crop.height) // 2
                canvas.paste(crop, (x_off, y_off))

                # plt.imshow(canvas)
                # plt.title('Video: ' + vid_id_test + ', Frame: ' + frame_idx + ', Cell_Id: ' + cell_id)
                # plt.show()

                # save cropped and resized image
                key = f"{vid_name}/{frame_idx}/{cell_id.zfill(3)}.jpg"
                keys.append(key)

                buffer = BytesIO()
                canvas.save(buffer, format="JPEG")  # or "PNG" if needed
                txn.put(key.encode("utf-8"), buffer.getvalue())
    txn.put(b"__keys__", b"\n".join(key.encode("utf-8") for key in keys))
    txn.put(b"__len__", str(len(keys)).encode("ascii"))

In [ ]:
for _, vid_name in tqdm(enumerate(video_folder_list), total=len(video_folder_list), ncols=100, position=0):

    # define paths for specific video folder
    org_video_folder = src_root / vid_name / 'images'
    dst_video_folder = dst_root / vid_name / 'images'

    if os.path.exists(dst_video_folder):
        continue
    else:
        dst_video_folder.mkdir(parents=True, exist_ok=True)
    
    try:
        annos_by_frame = annos_sorted[vid_name]
    except KeyError:
        print(vid_name)
        continue 

    # iterate over every image file in that folder
    for img_path in sorted(org_video_folder.glob("*.jpg")):
        # frame index is the last 3 digits in “…t086.jpg”
        frame_idx = img_path.stem[-3:]

        # skip if no cells in this frame
        if frame_idx not in annos_by_frame:
            print(img_path)
            continue
        
        img = Image.open(img_path).convert("RGB")

        for ann in annos_by_frame[frame_idx]:
            cell_id = str(ann['cell_id'])
            x_c, y_c, w, h = ann['bbox']

            # calculate values for cropping
            left   = int(x_c - w / 2)
            upper  = int(y_c - h / 2)
            right  = int(x_c + w / 2)
            lower  = int(y_c + h / 2)

            crop = img.crop((left, upper, right, lower))

            # resize image if it is larger than the Target size
            if crop.width > TARGET_SIZE or crop.height > TARGET_SIZE:
                scale = TARGET_SIZE / max(crop.width, crop.height)
                new_size = (int(crop.width  * scale), int(crop.height * scale))
                crop = crop.resize(new_size, Image.BICUBIC)

            # put image on black background of Target size
            canvas = Image.new("RGB", (TARGET_SIZE, TARGET_SIZE), (0, 0, 0))  # black background
            x_off = (TARGET_SIZE - crop.width)  // 2
            y_off = (TARGET_SIZE - crop.height) // 2
            canvas.paste(crop, (x_off, y_off))

            # plt.imshow(canvas)
            # plt.title('Video: ' + vid_id_test + ', Frame: ' + frame_idx + ', Cell_Id: ' + cell_id)
            # plt.show()

            # save cropped and resized image
            save_name  = f"v{vid_name}_f{frame_idx}_c{cell_id.zfill(3)}.jpg"
            canvas.save(dst_video_folder / save_name)

## Train Test Split

In [ ]:
videos_split = [{k: v[k] for k in ['id', 'name']} for v in videos if v['name'] not in missing_folders]

test_size = int(len(videos_split) * 0.2)
val_size = int(len(videos_split) * 0.8 * 0.2)

test_videos = []
val_videos = []

random.seed(42)

for i in range(test_size):
    test_videos.append(videos.pop(random.randint(0, len(videos) - 1)))

for i in range(val_size):
    val_videos.append(videos.pop(random.randint(0, len(videos) - 1)))

def sort(video):
    return video['id']

train_videos = videos
test_videos.sort(key=sort)
val_videos.sort(key=sort)

print(f"Train ({len(train_videos)}): {train_videos}")
print(f"Test ({len(test_videos)}):   {test_videos}")
print(f"Validation ({len(val_videos)}):   {val_videos}")

train_test_split = {'train': train_videos, 'test': test_videos, 'val': val_videos}

json.dump(train_test_split, open('train_test_split.json', 'w'))

## Calculate mean and std for Dataset

In [ ]:
class CellDataset(Dataset):
    def __init__(self, video_list, path_to_videos, transform):
        self.image_paths = []
        self.path_to_videos = path_to_videos
        self.transform = transform

        for video_dict in video_list:
            dst_path = path_to_videos / str(video_dict['name']) / 'images'
            self.image_paths += dst_path.glob('*.jpg')

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert("RGB")

        if self.transform:
            view1 = self.transform(img)
            view2 = self.transform(img)
            return view1, view2

        return img

In [ ]:
# Path to your JSON file
json_path = Path(base_path + '/train_test_split.json')

# Load the file
with open(json_path, 'r') as f:
    split_data = json.load(f)

# Access the train and test entries
train_list = split_data.get("train", [])
train_list

In [ ]:
# Simple transform to just convert to tensor
transform = transforms.ToTensor()
dataset = CellDataset(video_list=train_list, path_to_videos=dst_root, transform=transform)
loader = DataLoader(dataset, batch_size=32, shuffle=False)

In [ ]:
mean = 0.
std = 0.
nb_samples = 0.

for data in loader:
    img = data[0]  # assuming (batch, channels, height, width)
    batch_samples = img.size(0)
    img = img.view(batch_samples, img.size(1), -1)  # flatten HxW

    mean += img.mean(2).sum(0)
    std += img.std(2).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std /= nb_samples

print('Mean:', mean)
print('Std:', std)

In [ ]:
json.dump({'mean': mean.tolist(), 'std': std.tolist()}, open(base_path + '/mean_std.json', 'w'))

## Analyse Large Cells

In [ ]:
filtered_annos = [ann for ann in annos if ann['bbox'][2] > 400 or ann['bbox'][3] > 400]

# Step 1: Create a lookup from image_id to file_name
image_id_to_filename = {img["id"]: img["file_name"] for img in images}

# Step 2: Map filtered annotations to their filenames
for i in range(len(filtered_annos)):
    image_id = filtered_annos[i]['image_id']
    filename = image_id_to_filename.get(image_id, "UNKNOWN")
    filtered_annos[i]['filename'] = filename

In [ ]:
filtered_annos

In [ ]:
# How many full images to show
N = 20

for i, ann in enumerate(filtered_annos[:N]):
    file_path = os.path.join(src_root, ann["filename"])

    if not os.path.exists(file_path):
        print(f"Missing file: {file_path}")
        continue

    # Load full image
    img = Image.open(file_path).convert("RGB")

    # Show full image
    plt.imshow(img)
    plt.title(f"Frame {ann['time_step']} | Filename {ann['filename']}")
    plt.axis("off")
    plt.show()

In [ ]:
root_dir = Path(dst_root)
jpg_files = list(root_dir.rglob("*.jpg"))
print("Number of .jpg files:", len(jpg_files))